In [1]:
import pandas as pd
import numpy as np
import pickle, logging, spacy, sys, os, json, requests
import matplotlib.pyplot as plt

from helpers.classes import Collection
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
# DATASETS = os.listdir('datasets')

# with open('pickles/collection_20210624_194932.pkl', 'rb') as f:
#     collection = pickle.load(f)



# # I should completely remove the other bulletins
# # for now this has to do...
# bulletin_names = list(collection.bulletins.keys())
# # target bulletins are those with usable related datasets and main points
# target_bulletins = []
# dictionary = dict()
# for bulletin in bulletin_names:
#     if len(collection.bulletins.get(bulletin).get('main-points')) and len(collection.bulletins.get(bulletin).get('related-datasets')) > 0:
#         target_bulletins.append(bulletin)

In [3]:
from helpers.cloze_generation import generate_clozes_from_point, named_entity_answer_generator as ne_answer_generator, noun_phrase_answer_generator as np_answer_generator

df = pd.read_pickle('pickles/dataset_20210625_184837.pkl')
clozes_df = pd.read_json('pickles/clozes_20210715_212425.json')

In [4]:
df.head()

,bulletin,type,point,data
0,businessindustryandtrade/business/businessserv...,date_and_percent,"In 2019, approximate gross value added at basi...",[/businessindustryandtrade/business/businessse...
1,businessindustryandtrade/business/businessserv...,date_and_percent,"The non-financial services sector, which accou...",[/businessindustryandtrade/business/businessse...
2,businessindustryandtrade/business/businessserv...,date_and_percent,Total turnover and purchases of the UK non-fin...,[/businessindustryandtrade/business/businessse...
3,businessindustryandtrade/business/businessserv...,date_and_percent,"Out of the 12 UK regions, 8 regions experience...",[/businessindustryandtrade/business/businessse...
4,businessindustryandtrade/business/businessserv...,date_and_percent,"West Midlands, Yorkshire and The Humber, Scotl...",[/businessindustryandtrade/business/businessse...


## Bert

In [ ]:
from transformers import BertForMaskedLM, BertTokenizer, pipeline
from helpers.configs import CLOZE_MASKS
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# make sure to adjust RUN var 
# this section stores the most confident prediction
# and also keeps track of all unique entities 
RUN = False

if RUN:
    results = []
    entity_set = set()
    for row in tqdm(range(df.shape[0])):
        row_result = []
        clozes = [c for c in generate_clozes_from_point(df['point'][row], ne_answer_generator)]
        [entity_set.add((x.answer_text, x.answer_type)) for x in clozes]
        for cloze in clozes:
            result = check_model(model, tokenizer, cloze.cloze_text)

            answer_given = ''.join(result[0].get('token_str').split(' '))
            confidence = result[0].get('score')
            answer_true = cloze.answer_text

            # saves
            # prediction, confidence score, truth, dataframe row, cloze id
            row_result.append((answer_given, confidence, answer_true, row, cloze.cloze_id))

        results.append(row_result)

    with open('results/bert_base_check_model_july2.json', 'w') as f:
        json.dump(results, f)
    with open('results/bert_base_entity_set_july2.pickle', 'wb') as f:
        pickle.dump(entity_set, f)

    #  here I am saving the entities in a dictionary 
    #  with keys being each different entity category 
    #  such as MONEY, PERCENT and so on with values the unique terms found in our data

    categories = [x[1] for x in list(entity_set)]
    # construct keys
    entities = dict()
    entities = {f'{x}':[] for x in categories if x not in entities}
    # append only unique values
    [entities.get(x[1]).append(x[0]) for x in entity_set if x[0] not in entities.get(x[1])]

    with open('results/bert_base_entity_dictionary.json', 'w') as f:
        json.dump(entities, f)

In [5]:
# load back if you didn't run them

with open('results/bert_base_check_model_july2.json', 'r') as f:
    results = json.load(f)
with open('results/bert_base_entity_set_july2.pickle', 'rb') as f:
    entity_set = pickle.load(f)
with open('results/bert_base_entity_dictionary.json', 'r') as f:
    entities = json.load(f)

In [6]:
# I WASNT CONVERTING TO LOWER CASE!!!

# count_correct, count_wrong = 0, 0
# correct_preds, wrong_preds = [], []

# for row in results:
#     if len(row):
#         for entry in row:
#             if entry[0] == entry[2]:
#                 count_correct += 1
#                 correct_preds.append(entry[0])
#             else:
#                 count_wrong += 1
#                 wrong_preds.append((entry[0], entry[2]))

# print(f'Total Examples: {count_wrong + count_correct}')
# print(f'Correct: {count_correct}, Incorrect: {count_wrong}')
# print(f'Percentage Correct: {np.round( ((count_correct / (count_correct+ count_wrong) ) * 100), 3)}%')

Total Examples: 6184
Correct: 179, Incorrect: 6005
Percentage Correct: 2.895%


In [22]:
count_correct, count_wrong = 0, 0
correct_preds, wrong_preds = [], []

for row in results:
    if len(row):
        for entry in row:
            if str(entry[0]).lower() == str(entry[2]).lower():
                count_correct += 1
                correct_preds.append(str(entry[0]).lower())
            else:
                count_wrong += 1
                wrong_preds.append((str(entry[0]).lower(), str(entry[2]).lower()))

print(f'Total Examples: {count_wrong + count_correct}')
print(f'Correct: {count_correct}, Incorrect: {count_wrong}')
print(f'Percentage Correct: {np.round( ((count_correct / (count_correct+ count_wrong) ) * 100), 3)}%')

Total Examples: 6184
Correct: 654, Incorrect: 5530
Percentage Correct: 10.576%


In [23]:
print('BertBase Correct Predictions Snippet')
correct_preds[:50]

BertBase Correct Predictions Snippet


['2019',
 '2018',
 'year',
 'scotland',
 '2019',
 '2018',
 'london',
 '2018',
 'last year',
 'london',
 '2018',
 '2017',
 '2017',
 '2016',
 '2017',
 '2016',
 '2017',
 '2016',
 '2016',
 '2015',
 '2016',
 'london',
 '2016',
 'monthly',
 '2020',
 'london',
 '2018',
 'non-eu',
 'non-eu',
 '2018',
 '2017',
 'europe',
 'the united states',
 'london',
 '2018',
 'england',
 'the previous week',
 'the previous week',
 'hour',
 'nine',
 'around one',
 'five',
 'first',
 'first',
 'fye 2018',
 'fye 2018',
 'hours',
 'hours',
 '2016',
 '2015']

In [24]:
print('BertBase Incorrect Predictions Snippet')
wrong_preds[:50]

BertBase Incorrect Predictions Snippet


[('£88 usd. usd billion', '£1,313.9 billion'),
 ('£.bn.n', '£42.8 billion'),
 ('(ava ', '3.4%'),
 ('£,,,,, billion', '£4,101.5 billion'),
 ('£ in in billion in 2019 2020,', '£2,761.4 billion'),
 ('. sterling pounds billion sterling', '£70.5 billion'),
 ('(.% respectively', '1.7%'),
 ('£. respectivelybillion respectively', '£25.8 billion'),
 ('( respectively respectively respectively', '0.9%'),
 ('2016', '2018'),
 ('the region ireland', 'the south east'),
 ('.. up increase%', '£17.3 billion'),
 ('. £ level level billion', '£214.4 billion'),
 ('£ million expected expected', '8.8%'),
 ('2017', '2019'),
 ('</s>western', 'west midlands'),
 ('northern ireland', 'yorkshire'),
 ('london', 'scotland'),
 ('northern ireland', 'east midlands'),
 ('only', 'four'),
 (', england', 'west midlands'),
 ('. or or of%', '£2.5 billion'),
 ('(-% overall', '2.6%'),
 ('£. billion in 2016', '£94.5 billion'),
 ('£ billion billion', '£92 billion'),
 ('the entire period up', 'from late may 2021'),
 ('late 2022 le

## RoBERTA

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
model = RobertaForMaskedLM.from_pretrained('roberta-base')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
# make sure to adjust RUN var 
# this section stores the most confident prediction
# and also keeps track of all unique entities 
RUN = False

if RUN:
    results = []
    entity_set = set()
    for row in tqdm(range(df.shape[0])):
        row_result = []
        clozes = [c for c in generate_clozes_from_point(df['point'][row], ne_answer_generator)]
        [entity_set.add((x.answer_text, x.answer_type)) for x in clozes]
        for cloze in clozes:
            result = check_model(model, tokenizer, cloze.cloze_text)

            answer_given = ''.join(result[0].get('token_str').split(' '))
            confidence = result[0].get('score')
            answer_true = cloze.answer_text

            # saves
            # prediction, confidence score, truth, dataframe row, cloze id
            row_result.append((answer_given, confidence, answer_true, row, cloze.cloze_id))

        results.append(row_result)

    with open('results/roberta_base_check_model_july2.json', 'w') as f:
        json.dump(results, f)
    with open('results/roberta_base_entity_set_july2.pickle', 'wb') as f:
        pickle.dump(entity_set, f)

    #  here I am saving the entities in a dictionary 
    #  with keys being each different entity category 
    #  such as MONEY, PERCENT and so on with values the unique terms found in our data

    categories = [x[1] for x in list(entity_set)]
    # construct keys
    entities = dict()
    entities = {f'{x}':[] for x in categories if x not in entities}
    # append only unique values
    [entities.get(x[1]).append(x[0]) for x in entity_set if x[0] not in entities.get(x[1])]

    with open('results/roberta_base_entity_dictionary.json', 'w') as f:
        json.dump(entities, f)

In [9]:
# load back if you didn't run them

with open('results/roberta_base_check_model_july2.json', 'r') as f:
    results = json.load(f)
with open('results/roberta_base_entity_set_july2.pickle', 'rb') as f:
    entity_set = pickle.load(f)
with open('results/roberta_base_entity_dictionary.json', 'r') as f:
    entities = json.load(f)

In [10]:
count_correct, count_wrong = 0, 0
correct_preds, wrong_preds = [], []

for row in results:
    if len(row):
        for entry in row:
            if entry[0] == entry[2]:
                count_correct += 1
                correct_preds.append(entry[0])
            else:
                count_wrong += 1
                wrong_preds.append((entry[0], entry[2]))

print(f'Total Examples: {count_wrong + count_correct}')
print(f'Correct: {count_correct}, Incorrect: {count_wrong}')
print(f'Percentage Correct: {np.round( ((count_correct / (count_correct+ count_wrong) ) * 100), 3)}%')

Total Examples: 6184
Correct: 715, Incorrect: 5469
Percentage Correct: 11.562%


In [11]:
print('Roberta Base Correct Predictions Snippet')
correct_preds[:50]

Roberta Base Correct Predictions Snippet


['2019',
 '2018',
 'Yorkshire',
 'year',
 'annual',
 '2019',
 '2018',
 '2018',
 'London',
 '2018',
 '2017',
 '2017',
 '2016',
 '2017',
 '2016',
 '2017',
 '2016',
 '2016',
 '2015',
 '2016',
 'London',
 '2016',
 'annual',
 'annual',
 '2020',
 '2018',
 'Yorkshire',
 '2018',
 '2017',
 'Europe',
 '2018',
 'England',
 'Wales',
 'hour',
 'Nine',
 'five',
 'first',
 'first',
 'annual',
 'hours',
 'hours',
 '2016',
 '2015',
 '2016',
 '2015',
 '2016',
 '2016',
 '2016',
 '2016',
 '2016']

In [12]:
print('Roberta Base Incorrect Predictions Snippet')
wrong_preds[:50]

Roberta Base Incorrect Predictions Snippet


[('106', '£1,313.9 billion'),
 ('approximately', '£42.8 billion'),
 ('2019', '3.4%'),
 ('502', '£4,101.5 billion'),
 (',', '£2,761.4 billion'),
 ('2', '£70.5 billion'),
 ('1', '1.7%'),
 ('GDP', '£25.8 billion'),
 ('2', '0.9%'),
 ('2016', '2018'),
 ('Scotland', 'the South East'),
 ('revenue', '£17.3 billion'),
 ('2019', '£214.4 billion'),
 ('%', '8.8%'),
 ('2017', '2019'),
 ('England', 'West Midlands'),
 ('London', 'Scotland'),
 ('Wales', 'East Midlands'),
 ('only', 'four'),
 ('England', 'West Midlands'),
 ('half', '£2.5 billion'),
 ('2017', '2.6%'),
 ('114', '£94.5 billion'),
 ('89', '£92 billion'),
 ('up', 'from late May 2021'),
 ('date', 'early June 2021'),
 ('22', '87%'),
 ('2000', 'June 2020'),
 ('half', '31%'),
 ('highest', 'over 60%'),
 ('2015', 'January 2021'),
 ('Australia', 'Wales'),
 ('Australia', 'early May 2021'),
 ('19', '87%'),
 ('other', 'four'),
 ('the', 'late March to'),
 ('April', 'early May 2021'),
 ('It', 'Scotland'),
 ('2010', 'early November 2020'),
 ('Canada', 'N

## Electra

In [ ]:
from transformers import ElectraForMaskedLM, ElectraTokenizer
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
model = ElectraForMaskedLM.from_pretrained('google/electra-small-discriminator')

In [ ]:
# make sure to adjust RUN var 
# this section stores the most confident prediction
# and also keeps track of all unique entities 
RUN = False

if RUN:
    results = []
    entity_set = set()
    for row in tqdm(range(df.shape[0])):
        row_result = []
        clozes = [c for c in generate_clozes_from_point(df['point'][row], ne_answer_generator)]
        [entity_set.add((x.answer_text, x.answer_type)) for x in clozes]
        for cloze in clozes:
            result = check_model(model, tokenizer, cloze.cloze_text)

            answer_given = ''.join(result[0].get('token_str').split(' '))
            confidence = result[0].get('score')
            answer_true = cloze.answer_text

            # saves
            # prediction, confidence score, truth, dataframe row, cloze id
            row_result.append((answer_given, confidence, answer_true, row, cloze.cloze_id))

        results.append(row_result)

    with open('results/electra_base_check_model_july2.json', 'w') as f:
        json.dump(results, f)
    with open('results/electra_base_entity_set_july2.pickle', 'wb') as f:
        pickle.dump(entity_set, f)

    #  here I am saving the entities in a dictionary 
    #  with keys being each different entity category 
    #  such as MONEY, PERCENT and so on with values the unique terms found in our data

    categories = [x[1] for x in list(entity_set)]
    # construct keys
    entities = dict()
    entities = {f'{x}':[] for x in categories if x not in entities}
    # append only unique values
    [entities.get(x[1]).append(x[0]) for x in entity_set if x[0] not in entities.get(x[1])]

    with open('results/electra_base_entity_dictionary.json', 'w') as f:
        json.dump(entities, f)

In [13]:
# load back if you didn't run them

with open('results/electra_base_check_model_july2.json', 'r') as f:
    results = json.load(f)
with open('results/electra_base_entity_set_july2.pickle', 'rb') as f:
    entity_set = pickle.load(f)
with open('results/electra_base_entity_dictionary.json', 'r') as f:
    entities = json.load(f)

In [14]:
count_correct, count_wrong = 0, 0
correct_preds, wrong_preds = [], []

for row in results:
    if len(row):
        for entry in row:
            if entry[0] == entry[2]:
                count_correct += 1
                correct_preds.append(entry[0])
            else:
                count_wrong += 1
                wrong_preds.append((entry[0], entry[2]))

print(f'Total Examples: {count_wrong + count_correct}')
print(f'Correct: {count_correct}, Incorrect: {count_wrong}')
print(f'Percentage Correct: {np.round( ((count_correct / (count_correct+ count_wrong) ) * 100), 3)}%')

Total Examples: 6184
Correct: 0, Incorrect: 6184
Percentage Correct: 0.0%


## Albert

In [ ]:
from transformers import AlbertForMaskedLM, AlbertTokenizer
model = AlbertForMaskedLM.from_pretrained('albert-base-v2')
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

In [ ]:
# make sure to adjust RUN var 
# this section stores the most confident prediction
# and also keeps track of all unique entities 
RUN = False

if RUN:
    results = []
    entity_set = set()
    for row in tqdm(range(df.shape[0])):
        row_result = []
        clozes = [c for c in generate_clozes_from_point(df['point'][row], ne_answer_generator)]
        [entity_set.add((x.answer_text, x.answer_type)) for x in clozes]
        for cloze in clozes:
            result = check_model(model, tokenizer, cloze.cloze_text)

            answer_given = ''.join(result[0].get('token_str').split(' '))
            confidence = result[0].get('score')
            answer_true = cloze.answer_text

            # saves
            # prediction, confidence score, truth, dataframe row, cloze id
            row_result.append((answer_given, confidence, answer_true, row, cloze.cloze_id))

        results.append(row_result)

    with open('results/albert_base_check_model_july2.json', 'w') as f:
        json.dump(results, f)
    with open('results/albert_base_entity_set_july2.pickle', 'wb') as f:
        pickle.dump(entity_set, f)

    #  here I am saving the entities in a dictionary 
    #  with keys being each different entity category 
    #  such as MONEY, PERCENT and so on with values the unique terms found in our data

    categories = [x[1] for x in list(entity_set)]
    # construct keys
    entities = dict()
    entities = {f'{x}':[] for x in categories if x not in entities}
    # append only unique values
    [entities.get(x[1]).append(x[0]) for x in entity_set if x[0] not in entities.get(x[1])]

    with open('results/albert_base_entity_dictionary.json', 'w') as f:
        json.dump(entities, f)

In [15]:
# load back if you didn't run them

with open('results/albert_base_check_model_july2.json', 'r') as f:
    results = json.load(f)
with open('results/albert_base_entity_set_july2.pickle', 'rb') as f:
    entity_set = pickle.load(f)
with open('results/albert_base_entity_dictionary.json', 'r') as f:
    entities = json.load(f)

In [16]:
count_correct, count_wrong = 0, 0
correct_preds, wrong_preds = [], []

for row in results:
    if len(row):
        for entry in row:
            if entry[0] == entry[2]:
                count_correct += 1
                correct_preds.append(entry[0])
            else:
                count_wrong += 1
                wrong_preds.append((entry[0], entry[2]))

print(f'Total Examples: {count_wrong + count_correct}')
print(f'Correct: {count_correct}, Incorrect: {count_wrong}')
print(f'Percentage Correct: {np.round( ((count_correct / (count_correct+ count_wrong) ) * 100), 3)}%')

Total Examples: 6184
Correct: 216, Incorrect: 5968
Percentage Correct: 3.493%


In [17]:
print('Albert Base Correct Predictions Snippet')
correct_preds[:50]

Albert Base Correct Predictions Snippet


['year',
 '10%',
 '2019',
 '2018',
 '2018',
 '2018',
 '2017',
 '2017',
 '2016',
 '2017',
 '2016',
 '2017',
 '2016',
 '2016',
 '2015',
 'annual',
 'annual',
 '2018',
 '2018',
 'first',
 '2016',
 '2015',
 '2016',
 '2015',
 '2016',
 '2016',
 'year',
 'annually',
 'first',
 'quarterly',
 'annual',
 'annual',
 '2017',
 '2018',
 '50%',
 'annual',
 'annual',
 'annual',
 'annual',
 'first',
 'annual',
 'annual',
 '2019',
 '2018',
 '2018',
 '2018',
 '2017',
 '2018',
 '2016',
 '2017']

In [18]:
print('Albert Base Incorrect Predictions Snippet')
wrong_preds[:50]

Albert Base Incorrect Predictions Snippet


[('2018', '2019'),
 ('20%', '£1,313.9 billion'),
 ('10%', '£42.8 billion'),
 ('€', '3.4%'),
 ('2021', '2018'),
 ('$500,000', '£4,101.5 billion'),
 ('2018', '£2,761.4 billion'),
 ('2020', '£70.5 billion'),
 ('34%', '1.7%'),
 ('2020', '£25.8 billion'),
 ('34%', '0.9%'),
 ('2011', '2018'),
 ('uk', 'the South East'),
 ('2019', '£17.3 billion'),
 ('2020', '£214.4 billion'),
 ('gdp', '8.8%'),
 ('2010', '2019'),
 ('lancashire', 'West Midlands'),
 ('yorkshire', 'Yorkshire'),
 ('lincolnshire', 'Scotland'),
 ('wales', 'East Midlands'),
 ('largest', 'four'),
 ('cambridgeshire', 'West Midlands'),
 ('10%', '£2.5 billion'),
 ('€', '2.6%'),
 ('2009', '£94.5 billion'),
 ('2010', '£92 billion'),
 ('up', 'from late May 2021'),
 ('2021', 'early June 2021'),
 ('2015:', '87%'),
 ('2001', 'June 2020'),
 ('40%', '31%'),
 ('unchanged', 'over 60%'),
 ('2010', 'January 2021'),
 ('evalle', 'Wales'),
 ('wales', 'early May 2021'),
 ('joyah', '87%'),
 ('three', 'four'),
 ('the', 'late March to'),
 ('april', 'early 

# Multi-Token Language Model

In [19]:
with open('results/RobertaForMaskedLM_20210714_192239_results.json', 'r') as f:
    results = json.load(f)
with open('results/RobertaForMaskedLM_20210714_192239_entity_set.pickle', 'rb') as f:
    entity_set = pickle.load(f)
with open('results/RobertaForMaskedLM_20210714_192239_entity_dictionary.json', 'r') as f:
    entities = json.load(f)


from transformers import RobertaForMaskedLM, RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

In [20]:
count_correct, count_wrong = 0, 0
correct_preds, wrong_preds = [], []

for row in results:
    if len(row):
        for entry in row:
            # THIS IS AN ISSUE!!
            # IT SEEMS THAT THE MODEL PREDICTS A WHITESPACE AT THE START!!!
            if entry[0][0] == ' ':
                entry[0] = entry[0][1:]
            if entry[0] == entry[2]:
                count_correct += 1
                correct_preds.append(entry[0])
            else:
                count_wrong += 1
                wrong_preds.append((entry[0], entry[2]))

print(f'Total Examples: {count_wrong + count_correct}')
print(f'Correct: {count_correct}, Incorrect: {count_wrong}')
print(f'Percentage Correct: {np.round( ((count_correct / (count_correct+ count_wrong) ) * 100), 3)}%')

Total Examples: 6184
Correct: 654, Incorrect: 5530
Percentage Correct: 10.576%
